In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import time

In [2]:
# Setup paths untuk file lokal
base_dir = Path(".")
train_file = base_dir / "train70.csv"
test_file = base_dir / "test30.csv"

Mounted at /content/drive


In [3]:
# Path sudah di-setup di cell sebelumnya
print(f"Train file: {train_file}")
print(f"Test file: {test_file}")

In [4]:
def read_any_csv(path):
    try:
        return pd.read_csv(path, sep=None, engine='python')
    except Exception:
        return pd.read_csv(path)

In [5]:
def normalize_bool_like_fast(df):
    """Versi cepat dari normalize_bool_like"""
    df = df.loc[:, ~df.columns.str.startswith('Unnamed')]
    df = df.replace(to_replace=r'(?i)^\s*naan\s*$', value=np.nan, regex=True)
    df = df.apply(lambda col: col.astype(str).str.strip().str.lower()
                  if col.dtype == object else col)
    bool_map = {
        'true': 1, 't': 1, '1': 1, 'yes': 1, 'y': 1,
        'false': 0, 'f': 0, '0': 0, 'no': 0, 'n': 0
    }
    df = df.replace(bool_map)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df

In [6]:
def clean_df_fast(path):
    df = read_any_csv(path)
    before_shape = df.shape
    df = normalize_bool_like_fast(df)
    df = df.dropna(how='any')
    cols_check = df.columns.tolist()
    label_cols = [c for c in cols_check if c.lower() in ('label','label_binary','class')]
    feature_cols = [c for c in cols_check if c not in label_cols]
    if feature_cols:
        mask = (df[feature_cols] == 0).all(axis=1)
        df = df.loc[~mask]
    after_shape = df.shape
    return df, before_shape, after_shape

In [ ]:
start_time = time.time()

if not Path(train_file).exists() or not Path(test_file).exists():
    raise SystemExit("❌ File train70.csv atau test30.csv tidak ditemukan di Google Drive path.")

df_train, train_before, train_after = clean_df_fast(train_file)
df_test, test_before, test_after   = clean_df_fast(test_file)

print(f"Train shape: before={train_before} -> after={train_after}")
print(f"Test  shape: before={test_before} -> after={test_after}")

In [ ]:
alse))